In [14]:
import pandas as pd
import torch
from model_engine import Model
import pickle

In [10]:
model=Model()

In [11]:
model.load_state_dict(torch.load("Model_state.pth"))

C:\Users\joyma\AppData\Local\Temp\ipykernel_7948\2823781853.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("Model_state.pth"))


<All keys matched successfully>

In [85]:
input_data={
    "CreditScore":502,"Geography":"France",
    "Gender":"Female","Age":42,"Tenure":8,"Balance":159660.8,
    "NumOfProducts":3,"HasCrCard":1,
    "IsActiveMember":0,
    "EstimatedSalary":113931.57
}

In [86]:
with open("label_encoder.pkl","rb") as f:
    label_encoder=pickle.load(f)
with open("Standard_Scalar.pkl","rb") as f:
    standard_scalar=pickle.load(f)
with open("one_hot_encoder.pkl","rb") as f:
    OHE=pickle.load(f)

In [87]:
df=pd.DataFrame(input_data,index=[0])

In [88]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,502,France,Female,42,8,159660.8,3,1,0,113931.57


In [89]:
df["Gender"]=label_encoder.transform(df["Gender"])

In [90]:
new_df=OHE.transform(df[["Geography"]])

In [91]:
new_df=pd.DataFrame(new_df,columns=OHE.get_feature_names_out())

In [92]:
new_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [93]:
concat_df=pd.concat([df,new_df],axis=1)

In [94]:
concat_df=concat_df.drop(["Geography"],axis=1)

In [95]:
concat_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,502,0,42,8,159660.8,3,1,0,113931.57,1.0,0.0,0.0


In [96]:
x=standard_scalar.transform(concat_df)

In [97]:
x=torch.Tensor(x)

In [98]:
logits=model(x)
prediction=torch.round(torch.sigmoid(logits))

In [99]:
prediction.item()

1.0